In [ ]:
%matplotlib inline
%precision 3

import montecarlo
import plot
import metrics
from decimal import Decimal
import simulate
import pandas
import numpy
from pprint import pprint
import metrics
import withdrawal
import harvesting
import mortality

In [ ]:
r = simulate.withdrawals(montecarlo.LowYieldsAutoRegression(), years=30, withdraw=withdrawal.VPW)
plot.plot([n.withdraw_r for n in r], title='Withdrawals')
plot.plot([n.portfolio_post.value_r for n in r], title='Portfolio')
#plot.plot([n.returns for n in r], title='Returns', add_commas=False, zero_based=False)

In [ ]:
#pprint(['%.2f' % float(n.returns) for n in r[1:]])

In [ ]:
def harness(iters, sim):
    successes = 0
    incomes = pandas.Series(iters)
    for i in range(iters):
        sim_results = sim()
        incomes.loc[i] = float(metrics.cew([(n.withdraw_pct_orig * 100) + Decimal('.01') for n in sim_results]))
        if sim_results[-1].portfolio_pre.value_n > 0:
            successes += 1
    return (successes/iters, incomes)

def p_q(incomes):
    print('90th percentile income', incomes.quantile(.9))
    print('50th percentile income', incomes.quantile(.5))
    print('10th percentile income', incomes.quantile(.1))
    print('5th percentile income', incomes.quantile(.05))
    print('1st percentile income', incomes.quantile(.01))   

In [ ]:
iters = 50
EM = harness(iters, lambda: simulate.withdrawals(montecarlo.LowYieldsHighValuations(), years=30))
VPW = harness(iters, lambda: simulate.withdrawals(montecarlo.LowYieldsHighValuations(),harvesting=harvesting.N_60_RebalanceHarvesting, withdraw=withdrawal.VPW, years=30))
#ECM = harness(iters, lambda: simulate.withdrawals(montecarlo.LowYieldsAutoRegression(), withdraw=withdrawal.ECM, years=30))
VG = harness(iters, lambda: simulate.withdrawals(montecarlo.LowYieldsHighValuations(), withdraw=withdrawal.Vanguard, years=30))

In [ ]:
for i in (EM, VG, VPW):
    print('Success rate', i[0])
    p_q(i[1])
    print()

In [ ]:
def run_one(years, stock_pct, a=None):
    bond_pct = (1 - stock_pct)
    
    if not a:
        a = montecarlo.LowYieldsAutoRegression(initial_yield=.02697)
    
    returns = []
    
    for i, j in zip(range(years+1), a):
        stocks = float(j.stocks - j.inflation)
        bonds = float(j.bonds - j.inflation)
        total_return = (stocks * stock_pct) + (bonds * bond_pct)
        returns.append(total_return)
    return returns

In [ ]:
a = montecarlo.LowYieldsAutoRegression(initial_yield=.025, logging=True)
run_one(30, .6, a=a)
print('inflation', a.log['inflation'].mean())
print('stocks', a.log['stocks'].mean())
print('bonds', a.log['bonds'].mean())
print('-' * 65)
print(a.log)

In [ ]:
iters = 5000

s = pandas.Series(iters)

for i in range(iters):
    years = 30
    stock_pct = .6
    ssr = metrics.ssr([Decimal(n) for n in run_one(years, stock_pct)])
    s[i] = float(ssr)

In [ ]:
s.median()
s.quantile(.05)

In [ ]:
rates = pandas.Series(100)
for i in range(0, 100):
    q = s.quantile((100-i)/100)
    rates[i] = q * 100
rates.plot()

In [ ]:
survival_fn = mortality.make_mortality(mortality.NVSS_2011)
iters = 1000
successes = 0
for i in range(iters):
    lifespan = mortality.gen_lifespan(mortality.DEFAULT_COUPLE, survival_fn=survival_fn)
    sim_results = simulate.withdrawals(montecarlo.LowYieldsAutoRegression(),
                         harvesting=harvesting.make_rebalancer(.6),
                         withdraw=withdrawal.ConstantDollar,
                         years=lifespan)
    if sim_results[-1].portfolio_post.value_n > 0:
        successes += 1
print(successes/iters)

In [ ]:
survival_fn = mortality.make_mortality(mortality.NVSS_2011)
iters = 1000
successes = 0
failures = 0
years_to_failure = []
portfolio_at_10 = []
for i in range(iters):
    lifespan = mortality.gen_lifespan(mortality.DEFAULT_COUPLE, survival_fn=survival_fn)
    sim_results = simulate.withdrawals(montecarlo.LowYieldsAutoRegression(),
                         harvesting=harvesting.make_rebalancer(1),
                         withdraw=lambda p,h: withdrawal.ConstantDollar(p, h, rate=Decimal('.04')),
                         years=lifespan)

    # only count iterations where the initial 5 year return was at
    # least equal to what we've experienced.
    cumulative = pandas.Series([x.returns + 1 for x in sim_results[1:6]])
    if numpy.prod(cumulative) - 1 < 1.03660168511777: continue

    if sim_results[-1].portfolio_post.value_n > 0:
        successes += 1
    else:
        failures += 1
        years_to_failure.append(len(list(filter(lambda x: x.portfolio_r > 0, sim_results))))
        portfolio_at_10.append(sim_results[9].portfolio_r)
print(successes/(successes + failures))

In [ ]:
fail = pandas.Series(years_to_failure)
len(fail[fail < 20])

In [ ]:
p = pandas.Series(portfolio_at_10)

In [ ]:
p

In [ ]:
import numpy
import scipy.stats

iters = 5000
df = pandas.DataFrame(columns=numpy.arange(iters))

for i in range(iters):
    engine = montecarlo.LowYieldsAutoRegression()
    
    returns = []
    for j in range(5):
        returns.append(float(engine.random_year().stocks) + 1)
    series = pandas.Series(returns)
    
    df[i] = series

cumulative_returns = df.apply(lambda x: numpy.prod(x) - 1)

scipy.stats.percentileofscore(cumulative_returns, 1.03660168511777) # returns from 2011 - 2017 (inclusive)

In [ ]:
cumulative_returns.quantile(0.5)